1) Look at PPG of any or all subjects
2) label subjects with corrupted PPG signals, so they can be removed before stats later
3) Detect HR for each subject, label unusual HRs, save in the dataframe
4) Repeat 1-3 but with respiration data


In [1]:
import pandas as pd
import nibabel as nib
import numpy as np
from scipy.signal import find_peaks
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn.image import clean_img
from nilearn.image import new_img_like
from sklearn.metrics import r2_score
from nilearn.image import mean_img
import os


In [2]:
# # get subjects from subjects.txt:
# # Path to your subject list file
# subjects_file = r"D:\HCP_vessel_files\subjects.txt"

# # Read all IDs into a Python list
# with open(subjects_file, "r") as f:
#     subjects = [line.strip() for line in f if line.strip()]

# print("Loaded", len(subjects), "subjects:")

# # subjects = subjects[:10] # only do the first 10 subjects:
# subjects = subjects[10:20] # only do the first 10 subjects:


# print(subjects)  # show the first 10 to check


In [3]:
n_volumes = 1200  # HCP resting scan has 1200 volumes
TR = 0.72  # seconds
sampling_rate = 400  # Hz # Sampling rate (HCP standard)


In [4]:
# # Code to visualize the PPG data to check it. 
# # get subjects from subjects.txt:
# # Path to your subject list file
# subjects_file = r"D:\HCP_vessel_files\subjects.txt"
# # Read all IDs into a Python list
# with open(subjects_file, "r") as f:
#     subjects = [line.strip() for line in f if line.strip()]
# print("Loaded", len(subjects), "subjects:")
# # subjects = subjects[:10] 
# subjects = subjects[140:] 

# print(subjects)  # show the first 10 to check

# for subjectid in subjects:
#     print(f"subject {subjectid}")
#     hcp_physio_path = fr"D:\hcp_data\{subjectid}\unprocessed\3T\rfMRI_REST1_LR\LINKED_DATA\PHYSIO\{subjectid}_3T_rfMRI_REST1_LR_Physio_log.txt"
#     physio_data = np.loadtxt(hcp_physio_path)
#     print(f"✅ Loaded physio file, shape: {physio_data.shape}")     # Check shape
#     n_samples = physio_data.shape[0]

#     scan_duration = n_volumes * TR  # Total seconds
#     sampling_rate = n_samples / scan_duration     # Estimated sampling rate
#     print(f"✅ Estimated Sampling Rate: {sampling_rate:.2f} Hz")

#     pulse = physio_data[:, 2]
#     resp = physio_data[:, 1]
#     # Now you can process peaks 
#     peak_distance = int(0.55 * sampling_rate)  # Expected ~500-600 ms between heartbeats
#     peaks, _ = find_peaks(pulse, height=np.mean(pulse)*1.1, distance=peak_distance)

#     # Total time of scan (sec)
#     duration_sec = n_samples / sampling_rate  
#     # Beats per second
#     bps = len(peaks) / duration_sec  
#     # Convert to beats per minute
#     bpm = bps * 60  
#     print(f"✅ Mean heart rate: {bpm:.1f} BPM")


#     # time_vector = np.arange(len(pulse)) / sampling_rate
#     # plt.figure(figsize=(8, 6))
#     # index = 13000 # 345604 # 30000 # 4000
#     # plt.plot(time_vector[:index], pulse[:index], label='Pulse Ox signal')
#     # plt.plot(time_vector[peaks[(peaks < index)]], pulse[peaks[(peaks < index)]], 'rx', label='Detected peaks')
#     # plt.xlabel('Time (seconds)')
#     # plt.ylabel('PulseOx signal')
#     # plt.title(f'Pulse Ox signal {subjectid} with detected peaks')
#     # plt.legend()
#     # plt.show()

#     # Need to save each subject's Heart rate, PPG quality flag, 

# # Subjects with corrupted PPG from visually checking first 13000 time points (~30 seconds): 
# # Clearly corrupted: 158035, 192843, 620434, 729254, 770352, 
# # Not sure if corrupted: 102311, 107422, 113619, 127630, 150423, 151627, 155635, 162733, 316633, 771354, 
# # I think its not corrupted?: 103414, 110411, 113821, 113922, 137128, 138231, 142424, 167743, 
# # All others look good 


Loaded 190 subjects:
['592455', '594156', '598568', '601127', '613538', '620434', '622236', '638049', '672756', '683256', '685058', '690152', '725751', '729254', '729557', '732243', '742549', '748258', '748662', '756055', '770352', '771354', '779370', '816653', '826353', '826454', '833148', '837560', '837964', '856766', '857263', '859671', '861456', '865363', '872158', '877269', '887373', '889579', '896778', '898176', '901038', '901442', '912447', '930449', '937160', '965771', '983773', '987983', '992774', '994273']
subject 592455
✅ Loaded physio file, shape: (345604, 3)
✅ Estimated Sampling Rate: 400.00 Hz
✅ Mean heart rate: 69.4 BPM
subject 594156
✅ Loaded physio file, shape: (345605, 3)
✅ Estimated Sampling Rate: 400.01 Hz
✅ Mean heart rate: 67.8 BPM
subject 598568
✅ Loaded physio file, shape: (345605, 3)
✅ Estimated Sampling Rate: 400.01 Hz
✅ Mean heart rate: 73.6 BPM
subject 601127
✅ Loaded physio file, shape: (345605, 3)
✅ Estimated Sampling Rate: 400.01 Hz
✅ Mean heart rate: 63.

In [5]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

# --- Input paths ---
subjects_file = r"D:\HCP_vessel_files\subjects.txt"

# Load subject IDs
with open(subjects_file, "r") as f:
    subjects = [line.strip() for line in f if line.strip()]

# subjects = subjects[140:]  # adjust as needed

# --- Constants ---
TR = 0.72      # example repetition time in seconds (change to your TR)
n_volumes = 1200  # number of fMRI timepoints (adjust per your data)

# --- Initialize results list ---
results = []

for subjectid in subjects:
    print(f"Processing subject {subjectid} ...")

    hcp_physio_path = fr"D:\hcp_data\{subjectid}\unprocessed\3T\rfMRI_REST1_LR\LINKED_DATA\PHYSIO\{subjectid}_3T_rfMRI_REST1_LR_Physio_log.txt"

    try:
        physio_data = np.loadtxt(hcp_physio_path)
        n_samples = physio_data.shape[0]

        scan_duration = n_volumes * TR
        sampling_rate = n_samples / scan_duration

        pulse = physio_data[:, 2]
        peak_distance = int(0.55 * sampling_rate)
        peaks, _ = find_peaks(pulse, height=np.mean(pulse)*1.1, distance=peak_distance)

        duration_sec = n_samples / sampling_rate
        bpm = len(peaks) / duration_sec * 60

        print(f"✅ {subjectid}: {bpm:.1f} BPM ({sampling_rate:.2f} Hz)")
        results.append({"SubjectID": subjectid, "HeartRate_BPM": bpm, "SamplingRate_Hz": sampling_rate})

    except Exception as e:
        print(f"❌ Error loading subject {subjectid}: {e}")
        results.append({"SubjectID": subjectid, "HeartRate_BPM": np.nan, "SamplingRate_Hz": np.nan})

# --- Save to CSV ---
df = pd.DataFrame(results)
output_path = r"D:\hcp_processed\heart_rate_summary.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ Saved heart rate table to {output_path}")


Processing subject 100307 ...
✅ 100307: 76.9 BPM (400.00 Hz)
Processing subject 101309 ...
✅ 101309: 63.3 BPM (400.01 Hz)
Processing subject 102008 ...
✅ 102008: 44.9 BPM (400.01 Hz)
Processing subject 102311 ...
✅ 102311: 62.2 BPM (400.01 Hz)
Processing subject 103111 ...
✅ 103111: 89.4 BPM (400.01 Hz)
Processing subject 103414 ...
✅ 103414: 48.0 BPM (400.01 Hz)
Processing subject 103818 ...
✅ 103818: 91.8 BPM (400.00 Hz)
Processing subject 105014 ...
✅ 105014: 63.3 BPM (400.01 Hz)
Processing subject 107422 ...
✅ 107422: 52.0 BPM (400.01 Hz)
Processing subject 108828 ...
✅ 108828: 69.6 BPM (400.00 Hz)
Processing subject 109123 ...
✅ 109123: 58.3 BPM (400.00 Hz)
Processing subject 110411 ...
✅ 110411: 59.9 BPM (400.00 Hz)
Processing subject 111716 ...
✅ 111716: 69.5 BPM (400.01 Hz)
Processing subject 113619 ...
✅ 113619: 64.4 BPM (400.01 Hz)
Processing subject 113821 ...
✅ 113821: 70.5 BPM (400.01 Hz)
Processing subject 113922 ...
✅ 113922: 62.6 BPM (400.01 Hz)
Processing subject 11441